In [2]:
import cv2
import tkinter as tk
from tkinter import Label, Button
import datetime



# tkinter 윈도우 생성
root = tk.Tk()
root.title("ESP32-CAM Stream")

# Label 위젯을 생성하여 비디오 프레임을 표시
video_label = Label(root)
video_label.pack()

# 해상도를 표시할 Label 추가
resolution_label = Label(root, text="Resolution: N/A")
resolution_label.pack()

# 버튼 생성 (이미지 캡처)
capture_button = Button(root, text="Capture Image", command=lambda: capture_image(frame))
capture_button.pack()

# OpenCV를 사용하여 MJPEG 스트림을 가져옴
cap = cv2.VideoCapture(0)

frame = None  # 전역 변수로 초기화

def update_frame():
    global frame
    ret, frame = cap.read()
    if ret:
        # 해상도를 얻어 Label에 표시
        height, width, _ = frame.shape
        resolution_label.config(text=f"Resolution: {width}x{height}") 

In [4]:
import os
print(os.getcwd()) 

C:\Users\김효민\Desktop\python1


In [6]:
import cv2
import serial
import json
from ultralytics import YOLO
import threading 
import time

# 초기 시리얼 포트 설정99++++++++
arduino = None
serial_thread = None

# 아두이노 연결
arduino = serial.Serial("COM9", 9600)  # 포트 번호와 통신 속도 확인
print("파이썬과 아두이노가 연결되었습니다..\n")


# YOLOv8 모델 로드
model = YOLO("C:\\Users\\김효민\\Desktop\\python1\\best.pt")


# 비디오 스트림 열기
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("오류: 비디오 스트림을 열 수 없습니다.")
    exit()

# 시작 시간 기록


while True:
    ret, frame = cap.read()
    
    if not ret:
        print("프레임을 가져오지 못했습니다.")
        break
    
    # YOLOv8 객체 인식 실행
    results = model(frame)

    # 객체 카운팅 초기화
    # obj = {'spiders': 0}

    isTrue = False

    # 인식된 객체들 반복 처리
    for result in results:
        for box in result.boxes:
            # 신뢰도와 클래스 ID 가져오기
            conf = box.conf[0]
            class_id = int(box.cls[0].item())
            class_name = model.names[class_id]
                        
            if conf >= 0:                
                # 바운딩 박스 좌표 추출
                x1, y1, x2, y2 = map(int, box.xyxy[0])

                # 사각형 그리기
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                
                # 클래스명과 신뢰도 출력
                label = f'{class_name}: {conf:.2f}'
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                

            # 신뢰도 0.5 이상일 때 True로 설정
            if conf >= 0.5:
                isTrue = True


    # # 카메라가 10초 이상 가동되었으면 종료
    # if time.time() - start_time >= running_time:
    #     print("10초가 경과하여 카메라를 종료합니다.")
    #     break

    # print(obj)
    print(isTrue, arduino)

    6# 아두이노와 시리얼 통신
    if isTrue:
        json_data = json.dumps(isTrue) + '\n'
        print(json_data)
        arduino.write(json_data.encode('utf-8'))
        time.sleep(10)
    
    # 바운딩 박스와 클래스명이 포함된 프레임 표시
    cv2.imshow("CAM YOLOv8 Detection", frame)
    
    # 'q' 키를 눌러 루프 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    time.sleep(5)
        
# 비디오 캡처 객체 해제 및 OpenCV 창 닫기
cap.release()
cv2.destroyAllWindows()
arduino.close

파이썬과 아두이노가 연결되었습니다..


0: 480x640 (no detections), 234.7ms
Speed: 0.0ms preprocess, 234.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
False Serial<id=0x181e99fabc0, open=True>(port='COM9', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False)

0: 480x640 1 spiders, 200.5ms
Speed: 5.6ms preprocess, 200.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
True Serial<id=0x181e99fabc0, open=True>(port='COM9', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False)
true


0: 480x640 1 spiders, 194.6ms
Speed: 0.0ms preprocess, 194.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
True Serial<id=0x181e99fabc0, open=True>(port='COM9', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False)
true


0: 480x640 (no detections), 190.5ms
Speed: 0.0ms preprocess, 190.5ms inference, 0.0ms

<bound method Serial.close of Serial<id=0x181e99fabc0, open=True>(port='COM9', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False)>